In [1]:
%config IPCompleter.greedy=True


In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [3]:
import pandas as pd
def load_features(col):
    return pd.read_parquet("filtered_features/filtered_feature{}.gzip".format(col))

In [4]:
import pandas as pd

def load_test_features(col):
    return pd.read_parquet("filtered_test_features/filtered_test_feature{}.gzip".format(col))

In [5]:
NUM_OF_COL = 40

In [6]:
y = df_train['label']
y = y.values
y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [7]:
load_features(0).shape

(30336, 448)

In [8]:
from sklearn import decomposition
import numpy as np

XTrainAll = []
for col in range(NUM_OF_COL):
    tf = load_features(col)
    XTrainAll.append(tf)
XTrainAll = np.concatenate(XTrainAll,  axis=1)
XTrainAll.shape

(30336, 14564)

In [9]:
XTestAll = []
for col in range(NUM_OF_COL):
    tf = load_test_features(col)
    tf = tf.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
    XTestAll.append(tf)
XTestAll = np.concatenate(XTestAll,  axis=1)
XTestAll.shape

(10000, 14564)

In [10]:
df_test = pd.read_csv('sample_solution.csv')


In [11]:
y.shape

(30336,)

In [12]:
'''
from sklearn import decomposition

pca = decomposition.PCA(n_components=800)
pca.fit(XTrainAll)
XTrain = pca.transform(XTrainAll)
XTrain.shape
'''
XTrain = XTrainAll

In [13]:
'''
XTest = pca.transform(XTestAll)
XTest.shape
'''
XTest = XTestAll

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

gb_clf = GradientBoostingClassifier()
scores = cross_val_score(gb_clf, XTrain, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores

Accuracy: 0.96 (+/- 0.01)


array([0.96292024, 0.96192517, 0.96159552, 0.9627493 , 0.9513763 ])

In [15]:
gb_clf.fit(XTrain, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [16]:
probs = gb_clf.predict_proba(XTest)
YTest = probs[:, 0]
YTest

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:38: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


array([0.98682112, 0.98736849, 0.9695378 , ..., 0.98251769, 0.99032049,
       0.99099131])

In [17]:
np.max(YTest)

0.9912989596954785

In [18]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.986821
1,1,0.987368
2,2,0.969538
3,3,0.987739
4,4,0.987739
5,5,0.973983
6,6,0.985982
7,7,0.924997
8,8,0.981097
9,9,0.989277


In [19]:
df_test.to_csv('test.csv', index=False)